# Exercise sheet \#4
## Using APIs
### Exercise 1
Write a Python script which gets the list of astronauts who are currently in Space. To do so, you can use the [astro.json](http://api.open-notify.org/astros.json) OpenNotify API. 

In [1]:
import requests
response = requests.get("http://api.open-notify.org/astros.json")
data = response.json()
data

{'message': 'success',
 'people': [{'name': 'Jasmin Moghbeli', 'craft': 'ISS'},
  {'name': 'Andreas Mogensen', 'craft': 'ISS'},
  {'name': 'Satoshi Furukawa', 'craft': 'ISS'},
  {'name': 'Konstantin Borisov', 'craft': 'ISS'},
  {'name': 'Oleg Kononenko', 'craft': 'ISS'},
  {'name': 'Nikolai Chub', 'craft': 'ISS'},
  {'name': "Loral O'Hara", 'craft': 'ISS'}],
 'number': 7}

In [38]:
def get_astronaut_names():
    response = requests.get("http://api.open-notify.org/astros.json")
    data = response.json()
    names = [ dic.get('name', None) for dic in data['people'] ]
    return names

names = get_astronaut_names()
for _ in names:
    print('name: ' + _)

name: Jasmin Moghbeli
name: Andreas Mogensen
name: Satoshi Furukawa
name: Konstantin Borisov
name: Oleg Kononenko
name: Nikolai Chub
name: Loral O'Hara


### Exercise 2

#### Question 2.1
Write a Python program, which, for each astronaut found in Exercise 1, retrieves  their (English) wikipedia article and extract the article's summary and links.

In [3]:
!pip install wikipedia

In [4]:
import wikipedia 

In [5]:
def get_wikipedia_summary_and_link(name, lang='en'):
    wikipedia.set_lang('en')
    try:
        # search the wiki page
        page = wikipedia.page(name)

        # Extract the summary
        summary = page.summary

        # extract the links
        links = page.links

        return summary, links

    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for {name}: {e.options}")
        return None, None
    except wikipedia.exceptions.PageError:
        print(f"Page not found for {name}")
        return None, None

names = get_astronaut_names()
for name in names:
    summary, links = get_wikipedia_summary_and_link(name)
    if summary and links:
        print(f'name: {name}\nSummary: {summary}\nLinks: {links}\n')  # print the 10 first links


name: Jasmin Moghbeli
Summary: Jasmin Moghbeli (born June 24, 1983) is an American U.S. Marine Corps test pilot and NASA astronaut. She is a graduate of the Massachusetts Institute of Technology, Naval Postgraduate School, and Naval Test Pilot School. Moghbeli is mission commander for SpaceX’s Crew-7 and flight engineer aboard the International Space Station.
Links: ['Achievement Medal', 'Aerospace engineering', 'Air Medal', 'Andreas Mogensen', 'Arizona', 'Artemis program', 'Astronaut ranks and positions', "Bachelor's degree", 'Bachelor of Science', 'Bad Nauheim', 'Baldwin Senior High School (New York)', 'Bell AH-1Z Viper', 'Bell AH-1 SuperCobra', 'CBS Broadcasting', 'CBS News', 'California', 'Canadian Space Agency', 'China', 'Commendation Medal', 'Denmark', 'Expedition 69', 'Expedition 70', 'Extravehicular activity', 'Flight engineer', 'Francisco Rubio (astronaut)', 'Germany', 'ISS year-long mission', 'Information technology', 'Instagram (identifier)', 'International Space Station', '

name: Satoshi Furukawa
Summary: Satoshi Furukawa (古川 聡, Furukawa Satoshi, born April 4, 1964) is a Japanese surgeon and JAXA astronaut. Furukawa was assigned to the International Space Station as a flight engineer on long-duration missions Expedition 28/29, lifting off 7 June 2011 and returning 22 November 2011.
Links: ['14th CMAS Underwater Photography World Championship', '1973 Mount Gambier cave diving accident', '1992 cageless shark-diving expedition', '8A4-class ROUV', 'AAI underwater revolver', 'ABISMO', 'ADS amphibious rifle', 'AIDA Hellas', 'AIDA International', 'APS underwater rifle', 'AP Diving', 'ASM-DT amphibious rifle', 'ASRV Remora', 'Abrasive waterjet', 'Activated carbon', 'Adrian Biddle', 'Advanced Open Water Diver', 'Advanced SEAL Delivery System', 'Aerosinusitis', 'Aerospace Medical Association', 'Agnes Milowka', 'Aharon Solomons', 'Air-lock', 'Air embolism', 'Air line', 'Air spread', 'Airlift (dredging device)', 'Akihiko Hoshide', 'Albert A. Bühlmann', 'Albert D. Sto

name: Konstantin Borisov
Summary: Konstantin Sergeyevich Borisov (Russian: Константин Сергеевич Борисов; born 14 August 1984 in Smolensk) is a Russian cosmonaut who is currently flying on the SpaceX Crew-7 mission that launched on August 26, 2023.


Links: ['Andreas Mogensen', 'Astronaut', 'China', 'Denmark', 'Expedition 69', 'Expedition 70', 'ISS year-long mission', 'International Space Station', 'Japan', 'Jasmin Moghbeli', 'Jiang Xinlin', 'List of International Space Station expeditions', 'List of International Space Station spacewalks', 'List of Tiangong Space Station expeditions', 'List of human spaceflights in Tiangong Program', 'List of human spaceflights to the International Space Station', 'List of human spaceflights to the Tiangong space station', 'List of visitors to the International Space Station', "Loral O'Hara", 'Low Earth orbit', 'Moscow Aviation Institute', 'Nikolai Chub', 'Oleg Kononenko', 'Plekhanov Russian University of Economics', 'Roscosmos', 'Russia', 'Russian lan

#### Question 2.2
Extend your Python program so that it only keeps links that are pointing to wikipedia pages (in any language).

In [6]:
#my solution
def get_wikipedia_summary_and_link(name, lang='en'):
    wikipedia.set_lang('en')
    try:
        # Search the wiki page
        page = wikipedia.page(name)

        # Extract summary
        summary = page.summary
        
        # Extract links title and build wikipedia pages' url
        linked_titles = page.links
        wikipedia_links = ['https://en.wikipedia.org/wiki/' + title.replace(' ', '_') for title in linked_titles]

        return summary, wikipedia_links

    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for {name}: {e.options}")
        return None, None
    except wikipedia.exceptions.PageError:
        print(f"Page not found for {name}")
        return None, None

names = get_astronaut_names()
for name in names:
    summary, wikipedia_links = get_wikipedia_summary_and_link(name)
    if summary and wikipedia_links:
        print(f'Name: {name}\nSummary: {summary}\nWikipedia links: {wikipedia_links}\n')


Name: Jasmin Moghbeli
Summary: Jasmin Moghbeli (born June 24, 1983) is an American U.S. Marine Corps test pilot and NASA astronaut. She is a graduate of the Massachusetts Institute of Technology, Naval Postgraduate School, and Naval Test Pilot School. Moghbeli is mission commander for SpaceX’s Crew-7 and flight engineer aboard the International Space Station.
Wikipedia links: ['https://en.wikipedia.org/wiki/Achievement_Medal', 'https://en.wikipedia.org/wiki/Aerospace_engineering', 'https://en.wikipedia.org/wiki/Air_Medal', 'https://en.wikipedia.org/wiki/Andreas_Mogensen', 'https://en.wikipedia.org/wiki/Arizona', 'https://en.wikipedia.org/wiki/Artemis_program', 'https://en.wikipedia.org/wiki/Astronaut_ranks_and_positions', "https://en.wikipedia.org/wiki/Bachelor's_degree", 'https://en.wikipedia.org/wiki/Bachelor_of_Science', 'https://en.wikipedia.org/wiki/Bad_Nauheim', 'https://en.wikipedia.org/wiki/Baldwin_Senior_High_School_(New_York)', 'https://en.wikipedia.org/wiki/Bell_AH-1Z_Viper'

Name: Satoshi Furukawa
Summary: Satoshi Furukawa (古川 聡, Furukawa Satoshi, born April 4, 1964) is a Japanese surgeon and JAXA astronaut. Furukawa was assigned to the International Space Station as a flight engineer on long-duration missions Expedition 28/29, lifting off 7 June 2011 and returning 22 November 2011.
Wikipedia links: ['https://en.wikipedia.org/wiki/14th_CMAS_Underwater_Photography_World_Championship', 'https://en.wikipedia.org/wiki/1973_Mount_Gambier_cave_diving_accident', 'https://en.wikipedia.org/wiki/1992_cageless_shark-diving_expedition', 'https://en.wikipedia.org/wiki/8A4-class_ROUV', 'https://en.wikipedia.org/wiki/AAI_underwater_revolver', 'https://en.wikipedia.org/wiki/ABISMO', 'https://en.wikipedia.org/wiki/ADS_amphibious_rifle', 'https://en.wikipedia.org/wiki/AIDA_Hellas', 'https://en.wikipedia.org/wiki/AIDA_International', 'https://en.wikipedia.org/wiki/APS_underwater_rifle', 'https://en.wikipedia.org/wiki/AP_Diving', 'https://en.wikipedia.org/wiki/ASM-DT_amphibio

Name: Konstantin Borisov
Summary: Konstantin Sergeyevich Borisov (Russian: Константин Сергеевич Борисов; born 14 August 1984 in Smolensk) is a Russian cosmonaut who is currently flying on the SpaceX Crew-7 mission that launched on August 26, 2023.


Wikipedia links: ['https://en.wikipedia.org/wiki/Andreas_Mogensen', 'https://en.wikipedia.org/wiki/Astronaut', 'https://en.wikipedia.org/wiki/China', 'https://en.wikipedia.org/wiki/Denmark', 'https://en.wikipedia.org/wiki/Expedition_69', 'https://en.wikipedia.org/wiki/Expedition_70', 'https://en.wikipedia.org/wiki/ISS_year-long_mission', 'https://en.wikipedia.org/wiki/International_Space_Station', 'https://en.wikipedia.org/wiki/Japan', 'https://en.wikipedia.org/wiki/Jasmin_Moghbeli', 'https://en.wikipedia.org/wiki/Jiang_Xinlin', 'https://en.wikipedia.org/wiki/List_of_International_Space_Station_expeditions', 'https://en.wikipedia.org/wiki/List_of_International_Space_Station_spacewalks', 'https://en.wikipedia.org/wiki/List_of_Tiangong_Space_

Name: Nikolai Chub
Summary: Nikolay Aleksandrovich Chub (Russian: Николай Александрович Чуб; born 10 June 1984 in Novocherkassk, RSFSR, USSR, now Russia) is a cosmonaut selected by the Roscosmos space agency in 2012.
Wikipedia links: ['https://en.wikipedia.org/wiki/Andreas_Mogensen', 'https://en.wikipedia.org/wiki/Astronaut', 'https://en.wikipedia.org/wiki/Bibcode_(identifier)', 'https://en.wikipedia.org/wiki/China', 'https://en.wikipedia.org/wiki/Cosmonaut', 'https://en.wikipedia.org/wiki/Denmark', 'https://en.wikipedia.org/wiki/Doi_(identifier)', 'https://en.wikipedia.org/wiki/ESA_CAVES', 'https://en.wikipedia.org/wiki/European_Space_Agency', 'https://en.wikipedia.org/wiki/Expedition_68', 'https://en.wikipedia.org/wiki/Expedition_69', 'https://en.wikipedia.org/wiki/Expedition_70', 'https://en.wikipedia.org/wiki/Expedition_71', 'https://en.wikipedia.org/wiki/Extravehicular_activity', 'https://en.wikipedia.org/wiki/Hdl_(identifier)', 'https://en.wikipedia.org/wiki/ISSN_(identifier)', '

In [7]:
!pip install wptools

In [8]:
#correction 
import wptools

internal_links=[]

for alink in links[:10]:
    try:
        page=wikipedia.search(alink, results=1)
        #print page
        internal_links.append(alink)
    except wikipedia.exceptions.PageError as e:
        print(alink, e.options)
print(internal_links)

['Andreas Mogensen', 'Astronaut ranks and positions', 'Bachelor of Science', 'CBS Broadcasting', 'CBS News', 'Canadian Space Agency', 'China', 'Clements High School', 'DSV Alvin', 'Denmark']


In [9]:
type(internal_links[-1])

str

#### Question 2.3
Extend your Python program so that it processes these links as follows:
 - it retrieves the corresponding article and then extracts its references


In [10]:
# I have errors with this code
import sys

refs = []
for alink in internal_links:
    try:
        page=wikipedia.page(wikipedia.search(alink, results=1)[0])
        #print (page)
        refs.extend(page.references)
    except wikipedia.exceptions.PageError as e:
        print(e, file=sys.stderr)

print(refs)

C:\Users\Etudiant\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\Etudiant\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "CB" may refer to: 
CB postcode area
Cambodia
Cape Breton (disambiguation)
Centura București
Colegio Bolivar
Colwyn Bay
Province of Campobasso
České Budějovice
Chris Brown
Chris Bosh
Henry Campbell-Bannerman
Carte Bancaire
Christianssands Bryggeri
ScotAirways
Capacity building
Certification body
Chargeback
Convertible bond
Cell Broadcast
Citizens band radio
Closed beta
Cannabinoids
Circuit breaker
Columbium
Conduction band
Confirmation bias
Cumulonimbus cloud
CB-class midget submarine
Block coefficient
Large cruiser (CB)
Centre-back
Cornerback
Chicago Bears
Chicago Bulls
Cleveland Browns
Cincinnati Bengals
Chicago Blackhawks
Chief Baron of the Exchequer
Companion of The Most Honourable Order of the Bath
All pages with titles beginning with C. B.
2020 Singapore circuit breaker measures
leap year starting on Friday
Casus belli
Construction Battalion
Phish
Cone beam computed tomography
Internal Revenue Bulletin

In [11]:
import sys

refs = []
wikipedia.set_lang('en')
for alink in internal_links:
    try:
        search_results = wikipedia.search(alink, results=1)
        if search_results:
            page = wikipedia.page(search_results[0])
            refs.extend(page.references)
        else:
            print(f"No results found for {alink}", file=sys.stderr)

    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for {alink}: {e.options}", file=sys.stderr)
        

    except wikipedia.exceptions.PageError as e:
        print(e, file=sys.stderr)

print(refs)


Disambiguation error for CBS Broadcasting: ['CB postcode area', 'Cambodia', 'Cape Breton (disambiguation)', 'Centura București', 'Colegio Bolivar', 'Colwyn Bay', 'Province of Campobasso', 'České Budějovice', 'Chris Brown', 'Chris Bosh', 'Henry Campbell-Bannerman', 'Carte Bancaire', 'Christianssands Bryggeri', 'ScotAirways', 'Capacity building', 'Certification body', 'Chargeback', 'Convertible bond', 'Cell Broadcast', 'Citizens band radio', 'Closed beta', 'Cannabinoids', 'Circuit breaker', 'Columbium', 'Conduction band', 'Confirmation bias', 'Cumulonimbus cloud', 'CB-class midget submarine', 'Block coefficient', 'Large cruiser (CB)', 'Centre-back', 'Cornerback', 'Chicago Bears', 'Chicago Bulls', 'Cleveland Browns', 'Cincinnati Bengals', 'Chicago Blackhawks', 'Chief Baron of the Exchequer', 'Companion of The Most Honourable Order of the Bath', 'All pages with titles beginning with C. B.', '2020 Singapore circuit breaker measures', 'leap year starting on Friday', 'Casus belli', 'Construct

['http://www.esa.int/esaCP/SEMRO90OWUF_index_0.html', 'http://www.spacefacts.de/bios/international/english/morgensen_andreas.htm', 'http://www.rckonstanz.de/', 'http://en.rygaards.com/blog/former-rygaards-student-blasts-space', 'http://ing.dk/fokus/danmarks-foerste-astronaut', 'https://twitter.com/Astro_Andreas/status/643873701935644672', 'http://rumrejsen.dk/nyhed/k%C3%A6mpelyn', 'http://www.dmi.dk/nyheder/arkiv/nyheder-2015/09/saadan-fandt-dmi-kaempelynet-til-astro-andreas/', 'https://web.archive.org/web/20150916080348/https://twitter.com/Astro_Andreas/status/643873701935644672', 'http://rumrejsen.dk/andreas-mogensen-danskeren-vil-i-rummet', 'http://andreasmogensen.esa.int/', 'https://web.archive.org/web/20111020053708/http://videnskab.dk/profil/andreas-enevold-mogensen', 'https://viaf.org/viaf/8145601879001320813', 'https://web.archive.org/web/20151014015045/http://rumrejsen.dk/andreas-mogensen-danskeren-vil-i-rummet', 'https://web.archive.org/web/20150923133805/http://rumrejsen.dk/

#### Question 2.4
Extend your Python program to compute the average number of views for each astronaut's main article.

In [12]:
def get_num_views(name,lang='en'):
    wikipedia.set_lang('en')
    try:
        astronaut_page = wikipedia.page(name)
        
    except Exception as e:
        print(f"Error retrieving view data for {name}: {e}")
        return None


    views = wptools.page(name).get_more().data['views']
    return {'views':views}



name_astro = dict()
for name in names:
    name_astro[name] = get_num_views(name)

print(get_num_views)

en.wikipedia.org (querymore) Jasmin Moghbeli
Jasmin Moghbeli (en) data
{
  backlinks: <list(113)> {'pageid': 664, 'ns': 0, 'title': 'Astron...
  categories: <list(19)> Category:1983 births, Category:American p...
  contributors: 208
  files: <list(13)> File:Commons-logo.svg, File:Flag of Denmark.sv...
  languages: <list(25)> {'lang': 'ar', 'title': 'جاسمين مغبل'}, {'...
  pageid: 54253250
  requests: <list(1)> querymore
  title: Jasmin Moghbeli
  views: 1,225
}
en.wikipedia.org (querymore) Andreas Mogensen
en.wikipedia.org (querymore) 22874912 (&blcontinue=0|6961237)
en.wikipedia.org (querymore) 22874912 (&blcontinue=0|37699689)
Andreas Mogensen (en) data
{
  backlinks: <list(1362)> {'pageid': 664, 'ns': 0, 'title': 'Astro...
  categories: <list(11)> Category:1976 births, Category:21st-centu...
  contributors: 131
  files: <list(19)> File:Andreas Mogensen official portrait.jpg, F...
  languages: <list(25)> {'lang': 'ar', 'title': 'أندرياس موغنسن'},...
  pageid: 22874912
  requests: <li

<function get_num_views at 0x000001E979C5E430>


Loral O'Hara (en) data
{
  backlinks: <list(82)> {'pageid': 664, 'ns': 0, 'title': 'Astrona...
  categories: <list(13)> Category:1983 births, Category:21st-centu...
  contributors: 72
  files: <list(10)> File:Commons-logo.svg, File:Flag of Denmark.sv...
  languages: <list(16)> {'lang': 'ar', 'title': 'لورال أشلي أوهارا...
  pageid: 54253230
  requests: <list(1)> querymore
  title: Loral O'Hara
  views: 275
}


#### Question 2.5
Export the extracted information in a CSV file having the following fields:

`Astronaut's name ; Article's summary ; links separated by commas ; number of views`

In [13]:
import csv
import requests


def get_astronaut_data(name, lang='en'):
    summary, links = get_wikipedia_summary_and_link(name, lang='en')
    views = get_num_views(name)
    return summary, links, views

# Collect data 
astronaut_data = []
names = get_astronaut_names()
for name in names:
    summary, links, views = get_astronaut_data(name, lang='en')
    links_str = ', '.join(links) if links else 'No links'
    astronaut_data.append([name, summary, links_str, views])

# exportation in csv file
with open('astronauts_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Astronaut Name', 'Article Summary', 'Links', 'Number of Views'])
    for row in astronaut_data:
        writer.writerow(row)

print("Data exported to astronauts_data.csv")


en.wikipedia.org (querymore) Jasmin Moghbeli
Jasmin Moghbeli (en) data
{
  backlinks: <list(113)> {'pageid': 664, 'ns': 0, 'title': 'Astron...
  categories: <list(19)> Category:1983 births, Category:American p...
  contributors: 208
  files: <list(13)> File:Commons-logo.svg, File:Flag of Denmark.sv...
  languages: <list(25)> {'lang': 'ar', 'title': 'جاسمين مغبل'}, {'...
  pageid: 54253250
  requests: <list(1)> querymore
  title: Jasmin Moghbeli
  views: 1,225
}
en.wikipedia.org (querymore) Andreas Mogensen
en.wikipedia.org (querymore) 22874912 (&blcontinue=0|6961237)
en.wikipedia.org (querymore) 22874912 (&blcontinue=0|37699689)
Andreas Mogensen (en) data
{
  backlinks: <list(1362)> {'pageid': 664, 'ns': 0, 'title': 'Astro...
  categories: <list(11)> Category:1976 births, Category:21st-centu...
  contributors: 131
  files: <list(19)> File:Andreas Mogensen official portrait.jpg, F...
  languages: <list(25)> {'lang': 'ar', 'title': 'أندرياس موغنسن'},...
  pageid: 22874912
  requests: <li

Data exported to astronauts_data.csv


Loral O'Hara (en) data
{
  backlinks: <list(82)> {'pageid': 664, 'ns': 0, 'title': 'Astrona...
  categories: <list(13)> Category:1983 births, Category:21st-centu...
  contributors: 72
  files: <list(10)> File:Commons-logo.svg, File:Flag of Denmark.sv...
  languages: <list(16)> {'lang': 'ar', 'title': 'لورال أشلي أوهارا...
  pageid: 54253230
  requests: <list(1)> querymore
  title: Loral O'Hara
  views: 275
}


### Exercise 3
#### Question 3.1 
Using Wikipedia, compile the list of UEFA's Intertoto cup winners sorted by country.

In [14]:
import pandas as pd
uefa_table = wikipedia.page('List of UEFA Intertoto Cup winners').html().encode("utf-16")
df_club_country = pd.read_html(uefa_table)[1]
df_club_country

,Year,Nation,Winners,Score,Runners-up,Nation.1,Venue,Unnamed: 7
0,1995,France,Bordeaux,2–0,Karlsruher SC,Germany,"Wildparkstadion, Karlsruhe, Germany",NaN
1,1995,France,Bordeaux,2–2,Karlsruher SC,Germany,"Parc Lescure, Bordeaux, France",NaN
2,1995,Bordeaux won 4–2 on aggregate,Bordeaux won 4–2 on aggregate,Bordeaux won 4–2 on aggregate,Bordeaux won 4–2 on aggregate,Bordeaux won 4–2 on aggregate,Bordeaux won 4–2 on aggregate,Bordeaux won 4–2 on aggregate
3,1995,France,Strasbourg,1–1,Tirol Innsbruck,Austria,"Tivoli Neu, Innsbruck, Austria",NaN
4,1995,France,Strasbourg,6–1,Tirol Innsbruck,Austria,"Stade de la Meinau, Strasbourg, France",NaN
...,...,...,...,...,...,...,...,...
100,2007[b],Germany,Hamburger SV,4–0,Dacia Chișinău,Moldova,"HSH Nordbank Arena, Hamburg, Germany",NaN
101,2007[b],Hamburger SV won 5–1 on aggregate,Hamburger SV won 5–1 on aggregate,Hamburger SV won 5–1 on aggregate,Hamburger SV won 5–1 on aggregate,Hamburger SV won 5–1 on aggregate,Hamburger SV won 5–1 on aggregate,Hamburger SV won 5–1 on aggregate
102,2008[c],Portugal,Braga,2–0,Sivasspor,Turkey,"4 Eylül, Sivas, Turkey",NaN
103,2008[c],Portugal,Braga,3–0,Sivasspor,Turkey,"Estádio Municipal de Braga, Braga, Portugal",NaN


In [15]:
pairs = dict()

pairs = {row['Winners']: row['Nation'] for _, row in df_club_country.iterrows() if row['Nation'] != row['Winners'] and row['Winners'] not in pairs}


In [16]:
uefa_table2 = pd.read_html(uefa_table)[2]
uefa_table2

,Club,Titles,Runners-up,Years won,Years runner-up
0,Villarreal,2,1,"2003, 2004",2002
1,Hamburger SV,2,1,"2005, 2007",1999
2,VfB Stuttgart,2,0,"2000, 2002",—
3,Schalke 04,2,0,"2003, 2004",—
4,Karlsruher SC,1,1,1996,1995
5,Auxerre,1,1,1997,2000
6,Bologna,1,1,1998,2002
7,Valencia,1,1,1998,2005
8,Montpellier,1,1,1999,1997
9,Lille,1,1,2004,2002


In [17]:
winners_by_country = {}
for _, row in uefa_table2.iterrows():
    club = row['Club']
    if club in pairs:
        country = pairs[club]
        if country in winners_by_country:
            winners_by_country[country].append(club)
        else:
            winners_by_country[country] = [club]

# sort
for country in winners_by_country:
    winners_by_country[country].sort()


winners_by_country

{'Spain': ['Celta Vigo', 'Málaga', 'Valencia', 'Villarreal'],
 'Germany': ['Hamburger SV',
  'Karlsruher SC',
  'Schalke 04',
  'VfB Stuttgart',
  'Werder Bremen'],
 'France': ['Auxerre',
  'Bastia',
  'Bordeaux',
  'Guingamp',
  'Lens',
  'Lille',
  'Lyon',
  'Marseille',
  'Montpellier',
  'Paris Saint-Germain',
  'Strasbourg',
  'Troyes'],
 'Italy': ['Bologna', 'Juventus', 'Perugia', 'Udinese'],
 'England': ['Aston Villa', 'Fulham', 'Newcastle United', 'West Ham United'],
 'Denmark': ['Silkeborg'],
 'Portugal': ['Braga']}

#### Question 3.2
Compute the number of Intertoto winners per country.

In [18]:
titles_by_country = {}
for _, row in uefa_table2.iterrows():
    club = row['Club']
    titles = row['Titles']
    if club in pairs:
        country = pairs[club]
        if country in titles_by_country:
            titles_by_country[country] += titles
        else:
            titles_by_country[country] = titles


sorted_titles = sorted(titles_by_country.items(), key=lambda x: x[1], reverse=True)


sorted_titles

[('France', 12),
 ('Germany', 8),
 ('Spain', 5),
 ('Italy', 4),
 ('England', 4),
 ('Denmark', 1),
 ('Portugal', 1)]

### Exercise 4

Let's play with New York Times API! Register here to get your API access token: https://developer.nytimes.com/apis

#### Question 4.1

Extract for each astronaut articles in NYT mentioning them from the year 2022. Create a dictionay where by the astronauts name you could get information on number of articles published about them in 2022, links to the last 10 articles of 2022, and list of keywords which are associated with those articles, including how many times each keyword appeared. 

For example 

    {"John Smith": {"articles": ["nytimes.com/The_Best_Astronaut",
                                 "nytimes.com/Do_we_really_believe_in_space_travel"],
                   "num_articles": 2,
                   "keywords": {"space": 2, "celebrities": 1}}}
                   
<span style="color:red">ATTENTION!</span> Never publish your API access key on github or any other open website! It is only to be used by you. (Well, otherwise the API service could ban you :) )

In [37]:
import requests

base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"


api_key = 'qg1Oqiua7N6VQ8OcEsY0bftAH4clz0t7'


for astronaut in names:
    articles = []
    keywords = {}
    page = 0
    total_hits = 0

    while True:
        # Parameters for the API request
        params = {
            'q': astronaut,
            'api-key': api_key,
            'begin_date': "20220101",
            'end_date': "20221231",
            'sort': "newest",
            'page': page
        }

        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code != 200 or 'response' not in data:
            print("Error for :", astronaut)
            break

        docs = data['response']['docs']
        total_hits = data['response']['meta']['hits']

        for doc in docs:
            # Limit to the last 10 articles
            if len(articles) < 10:
                articles.append(doc['web_url'])

            for keyword in doc['keywords']:
                key = keyword['value']
                keywords[key] = keywords.get(key, 0) + 1

        page += 1
        if not docs or len(articles) >= 10:
            break

    astronaut_articles[astronaut] = {
        'num_articles': total_hits,
        'articles': articles,
        'keywords': keywords
    }

# Print the result
print(astronaut_articles)

## error :(

Error for : Nikolai Chub
Error for : Loral O'Hara
{'Astronaut1': {'num_articles': 0, 'articles': [], 'keywords': {}}, 'Astronaut2': {'num_articles': 0, 'articles': [], 'keywords': {}}, Ellipsis: {'num_articles': 22, 'articles': ['https://www.nytimes.com/2022/12/06/crosswords/daily-puzzle-2022-12-07.html', 'https://www.nytimes.com/2022/11/30/crosswords/daily-puzzle-2022-12-01.html', 'https://www.nytimes.com/2022/11/29/opinion/mike-pence-so-help-me-god.html', 'https://www.nytimes.com/2022/11/20/crosswords/daily-puzzle-2022-11-21.html', 'https://www.nytimes.com/2022/11/01/books/review/trespasses-louise-kennedy.html', 'https://www.nytimes.com/2022/10/24/crosswords/daily-puzzle-2022-10-25.html', 'https://www.nytimes.com/2022/10/23/crosswords/daily-puzzle-2022-10-24.html', 'https://www.nytimes.com/2022/09/25/crosswords/daily-puzzle-2022-09-26.html', 'https://www.nytimes.com/2022/09/10/crosswords/daily-puzzle-2022-09-11.html', 'https://www.nytimes.com/2022/09/05/crosswords/daily-puzzle-2022-0

In [26]:
response = requests.get(base_url, params=params)
data = response.json()
print(data)  

{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [], 'meta': {'hits': 0, 'offset': 0, 'time': 33}}}


#### Question 4.2. Bonus :) 

What happens when we want to extract links to all the articles and not just the last 10? Extend your program to iterate through pages of the query results :)